# Test registration on velodyne scans

In [1]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from copy import deepcopy
import open3d as o3d

import planeslam.io as io
from planeslam.scan import velo_pc_to_scan
from planeslam.general import NED_to_ENU, trajectory_plot_trace, pc_plot_trace
from planeslam.geometry.util import quat_to_R
from planeslam.registration import get_correspondences
from planeslam.point_cloud import velo_preprocess

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
np.set_printoptions(suppress=True)

In [3]:
# Read in point cloud data
pcpath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_20_2022', 'flightroom', 'run_7', 'pcs')
PCs = []
#len(os.listdir(pcpath))
#select_idxs = np.arange(0, 1000, 2)
select_idxs = np.arange(0, len(os.listdir(pcpath)), 5)
for i in select_idxs:  
    filename = os.path.join(pcpath, 'pc_'+str(i)+'.npy')
    PC = np.load(filename)
    PCs.append(PC)

In [4]:
# Read in pose data
posepath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_20_2022', 'flightroom', 'run_7', 'poses')
poses = []
for i in select_idxs:  
    filename = os.path.join(posepath, 'pose_'+str(i)+'.npy')
    pose = np.load(filename)
    poses.append(pose)

In [5]:
rover_rotations = np.zeros((3,3,len(poses)))
for i in range(len(poses)):
    rover_rotations[:,:,i] = quat_to_R(poses[i][3:])

In [6]:
# idx = 0
# fig = go.Figure(data=[pc_plot_trace(velo_preprocess(PCs[idx], poses[idx]))])
# fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
# fig.show()

In [7]:
# # Transform PCs to account for LiDAR offset
# offset_vec = np.array([0.09, 0.0, 0.0])

# for i in range(len(PCs)):
#     # Rotate offset by current pose
#     PCs[i] -= rover_rotations[:,:,0] @ offset_vec

In [8]:
# Plot ground-truth trajectory
rover_positions = np.asarray(poses)[:,:3]
gt_traj_trace = go.Scatter3d(x=rover_positions[:,0], y=rover_positions[:,1], z=rover_positions[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(rover_positions)))
gt_traj_trace.name = "Ground-truth"
fig = go.Figure(data=gt_traj_trace)
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [9]:
# Extract scans
num_scans = len(PCs)
scans = []
avg_extraction_time = 0
for i in range(num_scans):
    print(i)
    start_time = time.time()
    P = velo_preprocess(PCs[i], poses[i])
    scans.append(velo_pc_to_scan(P))
    scans[i].remove_small_planes(area_thresh=0.1)
    scans[i].reduce_inside(p2p_dist_thresh=0.1)
    avg_extraction_time += time.time() - start_time
avg_extraction_time /= num_scans
print(avg_extraction_time)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


c:\users\adam\navlab\lidar slam\planeslam\planeslam\mesh.py:173: RuntimeWarning:

invalid value encountered in true_divide



23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
0.04795526631305672


In [10]:
from planeslam.registration import robust_GN_register


start = 0

R_abs = quat_to_R(poses[start][3:])
t_abs = poses[start][:3].copy()
traj_est = np.zeros((num_scans-start, 3))
traj_est[0] = t_abs
traj_Rs = np.zeros((3, 3, num_scans-start))
traj_Rs[:,:,0] = R_abs
avg_runtime = 0

R_hats = []
t_hats = []

for i in range(1, num_scans-start):
    print("i = ", i)
    start_time = time.time()
    R_hat, t_hat = robust_GN_register(scans[i], scans[i-1], t_loss_thresh=0.1)
    t_abs += (R_abs @ t_hat).flatten()
    R_abs = R_hat @ R_abs
    avg_runtime += time.time() - start_time
    traj_est[i] = t_abs
    traj_Rs[:,:,i] = R_abs

    R_hats.append(R_hat)
    t_hats.append(t_hat)

    # R_1 = quat_to_R(poses[i-1][3:])
    # R_2 = quat_to_R(poses[i][3:])
    # R_gt = R_2 @ R_1.T

    # t_gt = poses[i][:3] - poses[i-1][:3]
    # print("R_hat: ", R_hat)
    # print("R_gt: ", R_gt)
    # print("t_hat: ", t_hat.flatten())
    # print("t_gt: ", t_gt)

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

i =  1
i =  2
i =  3
i =  4
i =  5
i =  6
i =  7
i =  8
i =  9
i =  10
i =  11
i =  12
i =  13
i =  14
i =  15
i =  16
i =  17
i =  18
i =  19
i =  20
i =  21
i =  22
i =  23
i =  24
i =  25
i =  26
i =  27
i =  28
i =  29
i =  30
i =  31
i =  32
i =  33
i =  34
i =  35
i =  36
i =  37
i =  38
i =  39
i =  40
i =  41
i =  42
i =  43
i =  44
i =  45
i =  46
i =  47
i =  48
i =  49
i =  50
i =  51
i =  52
i =  53
i =  54
i =  55
i =  56
i =  57
i =  58
i =  59
i =  60
i =  61
i =  62
i =  63
i =  64
i =  65
i =  66
i =  67
i =  68
i =  69
i =  70
i =  71
i =  72
i =  73
i =  74
i =  75
i =  76
i =  77
i =  78
i =  79
i =  80
i =  81
i =  82
i =  83
i =  84
i =  85
i =  86
i =  87
i =  88
i =  89
i =  90
i =  91
i =  92
i =  93
i =  94
i =  95
i =  96
i =  97
i =  98
i =  99
i =  100
i =  101
i =  102
i =  103
i =  104
i =  105
i =  106
i =  107
i =  108
i =  109
i =  110
i =  111
i =  112
i =  113
i =  114
i =  115
i =  116
i =  117
i =  118
i =  119
i =  120
i =  121
i =  122
i =  123
i

In [11]:
# Plot trajectories
est_traj_trace = go.Scatter3d(x=traj_est[:,0], y=traj_est[:,1], z=traj_est[:,2], hovertext=np.arange(len(traj_est)), marker=dict(size=5))
fig = go.Figure(data=[gt_traj_trace, est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [12]:
from graphslam.graph import Graph
from graphslam.vertex import Vertex
from graphslam.edge.edge_odometry import EdgeOdometry
from graphslam.pose.se3 import PoseSE3

from planeslam.geometry.util import R_to_quat

vertices = []
edges = []

# Add first vertex
p = PoseSE3(traj_est[0], R_to_quat(traj_Rs[:,:,0]))
v = Vertex(0, p)
vertices.append(v)

# For each scan
for i in range(1, num_scans):
    # Add new vertex
    p = PoseSE3(traj_est[i], R_to_quat(traj_Rs[:,:,i]))
    v = Vertex(i, p)
    vertices.append(v)

    # Add odometry edge
    information = np.eye(6)
    estimate = PoseSE3(t_hats[i-1], R_to_quat(R_hats[i-1]))
    estimate.normalize()
    e = EdgeOdometry([i-1, i], information, estimate)
    edges.append(e)

from planeslam.registration import loop_closure_register

# Add loop closure edges
loop_closures = [(0, 172)]#, (84, 244)]

for (i,j) in loop_closures:
    R_hat, t_hat = loop_closure_register(scans[j], scans[i], (traj_Rs[:,:,j], traj_est[j]), (traj_Rs[:,:,i], traj_est[i]), t_loss_thresh=0.1)
    information = np.eye(6)
    estimate = PoseSE3(t_hat, R_to_quat(R_hat))
    estimate.normalize()
    e = EdgeOdometry([i, j], information, estimate)
    edges.append(e)

g = Graph(edges, vertices)
g.optimize()


Iteration                chi^2        rel. change
---------                -----        -----------
        0               0.0292
        1               0.0001          -0.997333
        2               0.0000          -0.965316
        3               0.0000          -0.000012


In [13]:
positions = np.zeros((num_scans, 3))
for i, v in enumerate(g._vertices):
    positions[i] = v.pose.position

rotations = np.zeros((3, 3, num_scans))
for i, v in enumerate(g._vertices):
    rotations[:,:,i] = quat_to_R(v.pose.orientation)

Generate Map

In [14]:
# Initialize transformed scans
scans_transformed = num_scans * [None]
for i in range(num_scans):
    scans_transformed[i] = deepcopy(scans[i])
    scans_transformed[i].transform(rotations[:,:,i], positions[i])

In [15]:
merged = scans_transformed[0]

for s in scans_transformed[1:]:
    merged = merged.merge(s, dist_thresh=0.1)
    merged.reduce_inside(p2p_dist_thresh=0.1)
    merged.remove_small_planes(area_thresh=0.1)
    merged.fuse_edges(vertex_merge_thresh=0.1)

In [16]:
# Plot merge
fig = go.Figure(data=merged.plot_trace(colors=['blue']))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [17]:
del merged.planes[9]

In [18]:
del merged.planes[20]

In [19]:
map_final = merged

In [20]:
# Plot merge
fig = go.Figure(data=map_final.plot_trace(colors=['blue']))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data',
    xaxis=dict(title=dict(text='x (m)')), yaxis=dict(title=dict(text='y (m)')), zaxis=dict(title=dict(text='z (m)'))))
fig.show()

In [21]:
from scipy.spatial.transform import Rotation as R

r = R.from_euler('y', 1.0, degrees=True)
map_final.transform(r.as_matrix(), np.zeros(3))

In [48]:
from scipy.spatial.transform import Rotation as R

r = R.from_euler('z', 0.2, degrees=True)
map_final.transform(r.as_matrix(), np.zeros(3))

In [42]:
# Plot merge
fig = go.Figure(data=map_final.plot_trace(colors=['blue']))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data',
    xaxis=dict(title=dict(text='x (m)')), yaxis=dict(title=dict(text='y (m)')), zaxis=dict(title=dict(text='z (m)'))))
fig.show()

In [37]:
est_traj_trace = go.Scatter3d(x=positions[:,0], y=positions[:,1], z=positions[:,2], 
    marker=dict(size=5, color='orange'), name="Estimated")
gt_traj_trace = go.Scatter3d(x=rover_positions[:,0], y=rover_positions[:,1], z=rover_positions[:,2], 
    marker=dict(size=5, color='red'), name="Ground-truth")

In [49]:
# Plot merge
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=-1.75, z=1.75)
)

fig = go.Figure(data=map_final.plot_trace(colors=['blue'], showlegend=False)+[gt_traj_trace, est_traj_trace])
fig.update_layout(width=2500, height=1600, scene=dict(aspectmode='data',
    xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)),
    legend=dict(font=dict(size=30), yanchor="top", y=0.7, xanchor="left", x=0.12),
    scene_camera=camera)
fig.update_layout(legend= {'itemsizing': 'constant'})
fig.show()

In [ ]:
idx = 0
fig = go.Figure(data=[pc_plot_trace(velo_preprocess(PCs[idx], poses[idx]))]+scans[idx].plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [44]:
fig.write_image("../images/rover_map_and_traj.png", width=2500, height=1600)